In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install yfinance

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [40]:
import yfinance as yf

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "PFE",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "30m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2020-12-30 09:30:00-05:00,37.174999,37.217999,36.869999,37.099998,2945949
2020-12-30 10:00:00-05:00,37.099998,37.240002,37.009998,37.009998,1901644
2020-12-30 10:30:00-05:00,37.009998,37.139999,37.005001,37.090000,1440046
2020-12-30 11:00:00-05:00,37.080002,37.180000,37.070000,37.079899,996011
2020-12-30 11:30:00-05:00,37.075001,37.160000,37.020000,37.044998,1033359
...,...,...,...,...,...
2021-03-26 13:30:00-04:00,35.950001,36.080002,35.950001,36.064999,2182093
2021-03-26 14:00:00-04:00,36.063099,36.099998,36.009998,36.095001,1614327
2021-03-26 14:30:00-04:00,36.090000,36.130001,36.030102,36.099998,2005702


In [41]:
prices = data.reset_index()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype                           
---  ------    --------------  -----                           
 0   Datetime  780 non-null    datetime64[ns, America/New_York]
 1   Open      780 non-null    float64                         
 2   High      780 non-null    float64                         
 3   Low       780 non-null    float64                         
 4   Close     780 non-null    float64                         
 5   Volume    780 non-null    int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(1)
memory usage: 36.7 KB


In [42]:
prices['Datetime'] = prices['Datetime'].astype(str)

In [43]:
#remove -5:00
prices['Datetime'] = prices['Datetime'].str.replace('-05:00',"")
prices['Datetime'] = prices['Datetime'].str.replace('-04:00',"")

In [44]:
prices = prices[(prices['Datetime']>= '2020-12-22 09:30:00') & (prices['Datetime']<='2021-03-22 15:30:00')]
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-30 09:30:00,37.174999,37.217999,36.869999,37.099998,2945949
1,2020-12-30 10:00:00,37.099998,37.240002,37.009998,37.009998,1901644
2,2020-12-30 10:30:00,37.009998,37.139999,37.005001,37.090000,1440046
3,2020-12-30 11:00:00,37.080002,37.180000,37.070000,37.079899,996011
4,2020-12-30 11:30:00,37.075001,37.160000,37.020000,37.044998,1033359
...,...,...,...,...,...,...
723,2021-03-22 13:30:00,35.889999,35.919998,35.869999,35.900002,1309226
724,2021-03-22 14:00:00,35.895000,35.930000,35.880001,35.913300,930842
725,2021-03-22 14:30:00,35.919998,35.970001,35.910000,35.950001,969220
726,2021-03-22 15:00:00,35.950001,35.990002,35.930000,35.974998,1208629


In [45]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728 entries, 0 to 727
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  728 non-null    object 
 1   Open      728 non-null    float64
 2   High      728 non-null    float64
 3   Low       728 non-null    float64
 4   Close     728 non-null    float64
 5   Volume    728 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 39.8+ KB


In [46]:
sentiment = pd.read_csv('/content/drive/MyDrive/FA PROJECT 2021/Basic Regressors/PFE/60days_tweets.csv',lineterminator='\n')
sentiment.head()

,Datetime,Text,processed_tweet
0,2021-01-21 00:00:45+00:00,"Trump's Market Was So-So, Whichever Way You Cu...",trumps market was soso whichever way you cut i...
1,2021-01-21 00:05:52+00:00,"EU leaders convene amid vaccine delays, virus ...",eu leaders convene amid vaccine delays virus v...
2,2021-01-21 00:06:29+00:00,@spotgamma @StockBoardAsset #Israel: #Pfizerva...,israel pfizervaccine first dose less effective...
3,2021-01-21 00:07:51+00:00,@ASeizew @Kevin_W81 Think $BNTX and $PFE may d...,w81 think bntx and pfe may drop the acuitas a...
4,2021-01-21 00:12:32+00:00,Wednesday #NYSE Top 100 Volume Leaders\n#Black...,wednesday nyse top volume leaders blackberry l...


In [47]:
sentiment = sentiment.rename(columns={'Datetime': 'created_at'})

In [48]:
#remove +00:00
sentiment['created_at'] = sentiment['created_at'].str.slice(0,19)

In [49]:
sentiment.head()

,created_at,Text,processed_tweet
0,2021-01-21 00:00:45,"Trump's Market Was So-So, Whichever Way You Cu...",trumps market was soso whichever way you cut i...
1,2021-01-21 00:05:52,"EU leaders convene amid vaccine delays, virus ...",eu leaders convene amid vaccine delays virus v...
2,2021-01-21 00:06:29,@spotgamma @StockBoardAsset #Israel: #Pfizerva...,israel pfizervaccine first dose less effective...
3,2021-01-21 00:07:51,@ASeizew @Kevin_W81 Think $BNTX and $PFE may d...,w81 think bntx and pfe may drop the acuitas a...
4,2021-01-21 00:12:32,Wednesday #NYSE Top 100 Volume Leaders\n#Black...,wednesday nyse top volume leaders blackberry l...


In [50]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14475 entries, 0 to 14474
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   created_at       14475 non-null  object
 1   Text             14475 non-null  object
 2   processed_tweet  14475 non-null  object
dtypes: object(3)
memory usage: 339.4+ KB


In [51]:
# convert datetime and created at to same datetime object
prices['Datetime'] = pd.to_datetime(prices['Datetime'])
sentiment['created_at'] = pd.to_datetime(sentiment['created_at'])

In [52]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728 entries, 0 to 727
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  728 non-null    datetime64[ns]
 1   Open      728 non-null    float64       
 2   High      728 non-null    float64       
 3   Low       728 non-null    float64       
 4   Close     728 non-null    float64       
 5   Volume    728 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 39.8 KB


In [53]:
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-30 09:30:00,37.174999,37.217999,36.869999,37.099998,2945949
1,2020-12-30 10:00:00,37.099998,37.240002,37.009998,37.009998,1901644
2,2020-12-30 10:30:00,37.009998,37.139999,37.005001,37.090000,1440046
3,2020-12-30 11:00:00,37.080002,37.180000,37.070000,37.079899,996011
4,2020-12-30 11:30:00,37.075001,37.160000,37.020000,37.044998,1033359
...,...,...,...,...,...,...
723,2021-03-22 13:30:00,35.889999,35.919998,35.869999,35.900002,1309226
724,2021-03-22 14:00:00,35.895000,35.930000,35.880001,35.913300,930842
725,2021-03-22 14:30:00,35.919998,35.970001,35.910000,35.950001,969220
726,2021-03-22 15:00:00,35.950001,35.990002,35.930000,35.974998,1208629


In [54]:
# only close column is needed
prices = prices.drop(columns=['Open','High','Low','Volume'])

In [55]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14475 entries, 0 to 14474
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   created_at       14475 non-null  datetime64[ns]
 1   Text             14475 non-null  object        
 2   processed_tweet  14475 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 339.4+ KB


In [56]:
sentiment = sentiment.rename(columns={'created_at': 'Datetime'})
sentiment

,Datetime,Text,processed_tweet
0,2021-01-21 00:00:45,"Trump's Market Was So-So, Whichever Way You Cu...",trumps market was soso whichever way you cut i...
1,2021-01-21 00:05:52,"EU leaders convene amid vaccine delays, virus ...",eu leaders convene amid vaccine delays virus v...
2,2021-01-21 00:06:29,@spotgamma @StockBoardAsset #Israel: #Pfizerva...,israel pfizervaccine first dose less effective...
3,2021-01-21 00:07:51,@ASeizew @Kevin_W81 Think $BNTX and $PFE may d...,w81 think bntx and pfe may drop the acuitas a...
4,2021-01-21 00:12:32,Wednesday #NYSE Top 100 Volume Leaders\n#Black...,wednesday nyse top volume leaders blackberry l...
...,...,...,...
14470,2021-03-22 23:09:10,@MSollender While directed at autoimmune/infla...,while directed at autoimmuneinflammation this ...
14471,2021-03-22 23:13:02,AAPL*35\nPFE*145 https://t.co/g0t8FgImyq,aapl pfe
14472,2021-03-22 23:42:31,"Besides a great newsletter, the Slack communit...",besides a great newsletter the slack community...
14473,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...


In [57]:
# drop na of sentiment
sentiment = sentiment.dropna()

In [58]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14475 entries, 0 to 14474
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         14475 non-null  datetime64[ns]
 1   Text             14475 non-null  object        
 2   processed_tweet  14475 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 452.3+ KB


In [59]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  728 non-null    datetime64[ns]
 1   Close     728 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.1 KB


In [60]:
df_merge = pd.merge_asof(sentiment,prices,left_on='Datetime',right_on="Datetime",direction='backward')
df_merge

,Datetime,Text,processed_tweet,Close
0,2021-01-21 00:00:45,"Trump's Market Was So-So, Whichever Way You Cu...",trumps market was soso whichever way you cut i...,36.480000
1,2021-01-21 00:05:52,"EU leaders convene amid vaccine delays, virus ...",eu leaders convene amid vaccine delays virus v...,36.480000
2,2021-01-21 00:06:29,@spotgamma @StockBoardAsset #Israel: #Pfizerva...,israel pfizervaccine first dose less effective...,36.480000
3,2021-01-21 00:07:51,@ASeizew @Kevin_W81 Think $BNTX and $PFE may d...,w81 think bntx and pfe may drop the acuitas a...,36.480000
4,2021-01-21 00:12:32,Wednesday #NYSE Top 100 Volume Leaders\n#Black...,wednesday nyse top volume leaders blackberry l...,36.480000
...,...,...,...,...
14470,2021-03-22 23:09:10,@MSollender While directed at autoimmune/infla...,while directed at autoimmuneinflammation this ...,35.994999
14471,2021-03-22 23:13:02,AAPL*35\nPFE*145 https://t.co/g0t8FgImyq,aapl pfe,35.994999
14472,2021-03-22 23:42:31,"Besides a great newsletter, the Slack communit...",besides a great newsletter the slack community...,35.994999
14473,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...,35.994999


In [61]:
df_merge1 = df_merge[df_merge.Close.notnull()]
df_merge1

,Datetime,Text,processed_tweet,Close
0,2021-01-21 00:00:45,"Trump's Market Was So-So, Whichever Way You Cu...",trumps market was soso whichever way you cut i...,36.480000
1,2021-01-21 00:05:52,"EU leaders convene amid vaccine delays, virus ...",eu leaders convene amid vaccine delays virus v...,36.480000
2,2021-01-21 00:06:29,@spotgamma @StockBoardAsset #Israel: #Pfizerva...,israel pfizervaccine first dose less effective...,36.480000
3,2021-01-21 00:07:51,@ASeizew @Kevin_W81 Think $BNTX and $PFE may d...,w81 think bntx and pfe may drop the acuitas a...,36.480000
4,2021-01-21 00:12:32,Wednesday #NYSE Top 100 Volume Leaders\n#Black...,wednesday nyse top volume leaders blackberry l...,36.480000
...,...,...,...,...
14470,2021-03-22 23:09:10,@MSollender While directed at autoimmune/infla...,while directed at autoimmuneinflammation this ...,35.994999
14471,2021-03-22 23:13:02,AAPL*35\nPFE*145 https://t.co/g0t8FgImyq,aapl pfe,35.994999
14472,2021-03-22 23:42:31,"Besides a great newsletter, the Slack communit...",besides a great newsletter the slack community...,35.994999
14473,2021-03-22 23:45:57,$TGTX $ACIU $MSFT $AAPL $ZM $ZS $AMD $M $TSLA ...,tgtx aciu msft aapl zm zs amd m tsla crsp nflx...,35.994999


In [62]:
import datetime

df_final = pd.DataFrame()

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(2021, 1, 21), datetime.datetime(2021, 3, 23))
listofdates = []
for i in range (len(total_days)):
  date_time = total_days[i] + datetime.timedelta(minutes=30)
  date_time = date_time + datetime.timedelta(hours=15)
  end_time = date_time.strftime("%Y-%m-%d %H:%M:%S")

  date_time2 = total_days[i] + datetime.timedelta(hours=9)
  start_time = date_time2.strftime("%Y-%m-%d %H:%M:%S")
  
  listofdates.append([end_time, start_time])

# print(len(total_days))

for i in range(len(total_days)):
  df_final = df_final.append(df_merge1[(df_merge1['Datetime']<=listofdates[i][0]) & (df_merge1['Datetime']>=listofdates[i][1])])

In [63]:
df_final

,Datetime,Text,processed_tweet,Close
43,2021-01-21 09:36:57,S.Korea expects first batch of COVID-19 vaccin...,skorea expects first batch of covid vaccine fr...,36.209999
44,2021-01-21 09:40:57,"Dubai cancels non-essential surgery, live ente...",dubai cancels nonessential surgery live entert...,36.209999
45,2021-01-21 10:22:26,$PFE BIDEN TO USE PRODUCTION ACT TO BOOST VACC...,pfe biden to use production act to boost vacci...,36.259998
46,2021-01-21 10:29:16,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998
47,2021-01-21 10:29:57,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998
...,...,...,...,...
14280,2021-03-22 15:15:31,"Today Top Flow in #healthcare sector, Buy Flow...",today top flow in healthcare sector buy flow a...,35.974998
14281,2021-03-22 15:16:48,Irish PM warns of 'trouble' in vaccine supply ...,irish pm warns of trouble in vaccine supply ch...,35.974998
14282,2021-03-22 15:21:56,Unusual #Option Activity $PFE $GILD $BA $FB,unusual option activity pfe gild ba fb,35.974998
14283,2021-03-22 15:25:19,.@GovRonDeSantis @GovernorTomWolf @GovKemp @Go...,i explain here pharma is not liable for issue...,35.974998


In [64]:
df_final = df_final.reset_index(drop=True)
df_final

,Datetime,Text,processed_tweet,Close
0,2021-01-21 09:36:57,S.Korea expects first batch of COVID-19 vaccin...,skorea expects first batch of covid vaccine fr...,36.209999
1,2021-01-21 09:40:57,"Dubai cancels non-essential surgery, live ente...",dubai cancels nonessential surgery live entert...,36.209999
2,2021-01-21 10:22:26,$PFE BIDEN TO USE PRODUCTION ACT TO BOOST VACC...,pfe biden to use production act to boost vacci...,36.259998
3,2021-01-21 10:29:16,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998
4,2021-01-21 10:29:57,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998
...,...,...,...,...
4373,2021-03-22 15:15:31,"Today Top Flow in #healthcare sector, Buy Flow...",today top flow in healthcare sector buy flow a...,35.974998
4374,2021-03-22 15:16:48,Irish PM warns of 'trouble' in vaccine supply ...,irish pm warns of trouble in vaccine supply ch...,35.974998
4375,2021-03-22 15:21:56,Unusual #Option Activity $PFE $GILD $BA $FB,unusual option activity pfe gild ba fb,35.974998
4376,2021-03-22 15:25:19,.@GovRonDeSantis @GovernorTomWolf @GovKemp @Go...,i explain here pharma is not liable for issue...,35.974998


In [65]:
df_final["Comp"] = ''
df_final["Negative"] = ''
df_final["Neutral"] = ''
df_final["Positive"] = ''
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:36:57,S.Korea expects first batch of COVID-19 vaccin...,skorea expects first batch of covid vaccine fr...,36.209999,,,,
1,2021-01-21 09:40:57,"Dubai cancels non-essential surgery, live ente...",dubai cancels nonessential surgery live entert...,36.209999,,,,
2,2021-01-21 10:22:26,$PFE BIDEN TO USE PRODUCTION ACT TO BOOST VACC...,pfe biden to use production act to boost vacci...,36.259998,,,,
3,2021-01-21 10:29:16,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998,,,,
4,2021-01-21 10:29:57,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998,,,,
...,...,...,...,...,...,...,...,...
4373,2021-03-22 15:15:31,"Today Top Flow in #healthcare sector, Buy Flow...",today top flow in healthcare sector buy flow a...,35.974998,,,,
4374,2021-03-22 15:16:48,Irish PM warns of 'trouble' in vaccine supply ...,irish pm warns of trouble in vaccine supply ch...,35.974998,,,,
4375,2021-03-22 15:21:56,Unusual #Option Activity $PFE $GILD $BA $FB,unusual option activity pfe gild ba fb,35.974998,,,,
4376,2021-03-22 15:25:19,.@GovRonDeSantis @GovernorTomWolf @GovKemp @Go...,i explain here pharma is not liable for issue...,35.974998,,,,


In [66]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4378 entries, 0 to 4377
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         4378 non-null   datetime64[ns]
 1   Text             4378 non-null   object        
 2   processed_tweet  4378 non-null   object        
 3   Close            4378 non-null   float64       
 4   Comp             4378 non-null   object        
 5   Negative         4378 non-null   object        
 6   Neutral          4378 non-null   object        
 7   Positive         4378 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 273.8+ KB


In [67]:
df_final['processed_tweet'] = df_final['processed_tweet'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4378 entries, 0 to 4377
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         4378 non-null   datetime64[ns]
 1   Text             4378 non-null   object        
 2   processed_tweet  4378 non-null   object        
 3   Close            4378 non-null   float64       
 4   Comp             4378 non-null   object        
 5   Negative         4378 non-null   object        
 6   Neutral          4378 non-null   object        
 7   Positive         4378 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 273.8+ KB


In [68]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df_final['Comp'] = [analyzer.polarity_scores(x)['compound'] for x in df_final['processed_tweet']]
df_final['Negative'] = [analyzer.polarity_scores(x)['neg'] for x in df_final['processed_tweet']]
df_final['Neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df_final['processed_tweet']]
df_final['Positive'] = [analyzer.polarity_scores(x)['pos'] for x in df_final['processed_tweet']]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [69]:
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:36:57,S.Korea expects first batch of COVID-19 vaccin...,skorea expects first batch of covid vaccine fr...,36.209999,0.0000,0.000,1.000,0.000
1,2021-01-21 09:40:57,"Dubai cancels non-essential surgery, live ente...",dubai cancels nonessential surgery live entert...,36.209999,0.2263,0.139,0.657,0.204
2,2021-01-21 10:22:26,$PFE BIDEN TO USE PRODUCTION ACT TO BOOST VACC...,pfe biden to use production act to boost vacci...,36.259998,0.4019,0.000,0.803,0.197
3,2021-01-21 10:29:16,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998,0.3612,0.000,0.839,0.161
4,2021-01-21 10:29:57,Germany’s Chancellor Merkel: Pfizer has promis...,germanys chancellor merkel pfizer has promised...,36.259998,0.3612,0.000,0.839,0.161
...,...,...,...,...,...,...,...,...
4373,2021-03-22 15:15:31,"Today Top Flow in #healthcare sector, Buy Flow...",today top flow in healthcare sector buy flow a...,35.974998,0.2023,0.000,0.952,0.048
4374,2021-03-22 15:16:48,Irish PM warns of 'trouble' in vaccine supply ...,irish pm warns of trouble in vaccine supply ch...,35.974998,-0.6124,0.300,0.700,0.000
4375,2021-03-22 15:21:56,Unusual #Option Activity $PFE $GILD $BA $FB,unusual option activity pfe gild ba fb,35.974998,0.0000,0.000,1.000,0.000
4376,2021-03-22 15:25:19,.@GovRonDeSantis @GovernorTomWolf @GovKemp @Go...,i explain here pharma is not liable for issue...,35.974998,0.0000,0.000,1.000,0.000


In [70]:
df_final = df_final.groupby(df_final.Datetime.dt.floor('30min')).mean()

In [71]:
df_final = df_final.drop(columns=['Negative','Neutral','Positive'])
df_final

,Close,Comp
Datetime,,
2021-01-21 09:30:00,36.209999,0.113150
2021-01-21 10:00:00,36.259998,0.374767
2021-01-21 10:30:00,36.285000,0.203680
2021-01-21 11:00:00,36.379101,0.339467
2021-01-21 11:30:00,36.305000,-0.045900
...,...,...
2021-03-22 13:00:00,35.889999,0.309813
2021-03-22 13:30:00,35.900002,0.212863
2021-03-22 14:00:00,35.913300,-0.028017


In [72]:
df_final.to_csv('/content/drive/MyDrive/FA PROJECT 2021/Basic Regressors/PFE/final.csv')